In [ ]:
"""
Created on Fri May 21 14:29 2021

This is a basic script to apply some parameters (e.g. originally proposed ones or tuned ones) 
to the different parameterisations resulting into 2D and 1D information about the melt

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from tqdm.notebook import trange, tqdm
import time
import assess_param_funcs.melt_functions as meltf
import assess_param_funcs.useful_functions as uf
from assess_param_funcs.constants import *
import os, glob

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path='/bettik/burgardc/'

In [ ]:
nemo_run = 'OPM021'

In [ ]:
inputpath_data=home_path+'/DATA/BASAL_MELT_PARAM/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = home_path+'/DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = home_path+'/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
outputpath_simple = home_path+'/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
outputpath_simple_all = home_path+'/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'
inputpath_plumes = home_path+'/DATA/BASAL_MELT_PARAM/interim/PLUMES/nemo_5km_'+nemo_run+'/'
outputpath_plumes = home_path+'/DATA/BASAL_MELT_PARAM/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = home_path+'/DATA/BASAL_MELT_PARAM/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_boxes = home_path+'/DATA/BASAL_MELT_PARAM/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = home_path+'/DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath = home_path+'/DATA/BASAL_MELT_PARAM/interim/'
plot_path = home_path+'/PLOTS/first-look/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)

file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')
file_TS = file_TS_orig.sel(Nisf=large_isf).sel(profile_domain=[50,1000]).isel(time=range(40,70))

In [ ]:
lon = file_isf.longitude
lat = file_isf.latitude

In [ ]:
file_isf_conc = file_conc_cut['isfdraft_conc']

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos

In [ ]:
isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')

FUNCTIONS

In [ ]:
param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

In [ ]:
geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
geometry_info_1D = param_var_of_int_1D

In [ ]:
# mean sin of local slope over whole Antarctica
np.sin(uf.weighted_mean(geometry_info_2D['alpha'].sel(option='appenB'),['x','y'],file_isf_conc)) 

SIMPLE PARAMS

In [ ]:
# best-estimate tuned parameters
gammas_file_simple = xr.open_dataset(outputpath_simple_all+'gammas_simple_ALL.nc')

#######
# Gammas from Favier et al., 2019 (to be used with U_param = False)
favier_gammas = xr.DataArray(np.array([2.030e-5,99.32e-5,132.9e-5]),dims='param')
favier_gammas = favier_gammas.assign_coords({'param': ['linear_local', 'quadratic_local', 'quadratic_mixed_mean']})
#######
# Gammas from Lipscomb et al., 2021 (// Jourdain et al. 2020) (to be used with U_param = False)
lipscomb_values_mpery = xr.DataArray(np.array([[1.11*10**4,1.44*10**4,2.06*10**6],
                                              [4.95*10**4,1.59*10**5,5.37*10**6]]), dims={'calib': ['MeanAnt','PIGL'], 'param': ['quadratic_local', 'quadratic_mixed_mean', 'quadratic_mixed_mean_locslope']})
lipscomb_values_mpery = lipscomb_values_mpery.assign_coords({'calib': ['MeanAnt','PIGL'], 'param': ['quadratic_local', 'quadratic_mixed_mean', 'quadratic_mixed_locslope']})

lipscomb_values_mpers = lipscomb_values_mpery/yearinsec

In [ ]:
simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_local_cavslope', 
                      'quadratic_mixed_mean', 'quadratic_mixed_locslope','quadratic_mixed_cavslope'] 

nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for mparam in simple_param_types:
        
    for gamma_choice in ['tuned','favier','lipscomb']:
        if gamma_choice == 'tuned':
            gamma = gammas_file_simple['slope'].sel(param=mparam)
            ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=True)

            ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')
            ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')
                
        elif gamma_choice == 'favier':
            if mparam in favier_gammas.param.values:
                gamma = favier_gammas.sel(param=mparam)
                ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=False)
                
                ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')
                ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')
                
        elif gamma_choice == 'lipscomb':
            if mparam in lipscomb_values_mpers.param.values:
                for ccalib in lipscomb_values_mpers.calib:
                    gamma = lipscomb_values_mpers.sel(param=mparam, calib=ccalib)
                    ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=False)  

                    ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+str(ccalib.values)+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')
                    ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+str(ccalib.values)+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')

In [ ]:
simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_mixed_mean', 'quadratic_mixed_locslope','quadratic_mixed_cavslope']
for mparam in simple_param_types:
     if mparam in favier_gammas.param:
            print(mparam)

SIMPLE BOTTOM ENTRY

In [ ]:
simple_param_types = ['linear_local_bottom', 'quadratic_local_bottom', 'quadratic_local_locslope_bottom', 'quadratic_local_cavslope_bottom', 
                      'quadratic_mixed_mean_bottom', 'quadratic_mixed_locslope_bottom','quadratic_mixed_cavslope_bottom'] 

In [ ]:
gammas_file_simple_bottom = xr.open_dataset(outputpath_simple_all+'linreg_experiment_bottom_results.nc')


In [ ]:
simple_param_types = ['linear_local_bottom', 'quadratic_local_bottom', 'quadratic_local_locslope_bottom', 'quadratic_local_cavslope_bottom', 
                      'quadratic_mixed_mean_bottom', 'quadratic_mixed_locslope_bottom','quadratic_mixed_cavslope_bottom'] 

nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for mparam in simple_param_types:
        
    #for gamma_choice in ['tuned','favier','lipscomb']:
    for gamma_choice in ['tuned']:
        if gamma_choice == 'tuned':
            gamma = gammas_file_simple_bottom['slope'].sel(param=mparam)
            ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=True)

            ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')
            ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')

PLUMES

In [ ]:
tuned_plume_gammas = xr.DataArray(np.array([[2.8e-4,42.2e-4],
                                              [1.3e-4,1e3]]), dims={'param': ['lazero19_2','lazero19_modif2'], 'profile_domain': np.array([50,1000])})
tuned_plume_gammas = tuned_plume_gammas.assign_coords({'param': ['lazero19_2','lazero19_modif2'], 'profile_domain': np.array([50,1000])})

tuned_plume_E0s = xr.DataArray(np.array([[4.2e-2,0.34e-2],
                                  [7.6e-2,0.14e-2]]), dims={'param': ['lazero19_2','lazero19_modif2'], 'profile_domain': np.array([50,1000])})
tuned_plume_E0s = tuned_plume_E0s.assign_coords({'param': ['lazero19_2','lazero19_modif2'], 'profile_domain': np.array([50,1000])})

In [ ]:
plume_param_types = ['lazero19_2','lazero19_modif2'] 
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for mparam in plume_param_types:
    for gamma_choice in ['orig','tuned']: #'orig','tuned'
        
        if gamma_choice == 'tuned':
            
            if mparam == 'lazero19_modif2':
                
                gamma = tuned_plume_gammas.sel(param=mparam)
                E0 = tuned_plume_E0s.sel(param=mparam)

            elif mparam == 'lazero19_2':
                
                print('entering _2')
                gamma = tuned_plume_gammas.sel(param=mparam)
                E0 = tuned_plume_E0s.sel(param=mparam)
                
            ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, E0=E0, verbose=True)
            
        elif gamma_choice == 'orig':
            
            print('entering orig')
            
            gamma = gamma_eff_T_lazero
            E0 = E0_lazero
                
            ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, E0=E0, verbose=True)
                
        ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')
        ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')

BOXES

In [ ]:
tuned_box_gammas = xr.DataArray(np.array([[[0.39e-5,0.41e-5,0.44e-5,0.39e-5],
                                              [0.39e-5,0.41e-5,0.44e-5,0.39e-5]],
                                         [[0.51e-5,0.73e-5,0.92e-5,0.63e-5],
                                              [0.51e-5,0.73e-5,0.92e-5,0.63e-5]]]
                                        ), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_gammas = tuned_box_gammas.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})


tuned_box_Cs = xr.DataArray(np.array([[[16.5e6,18.0e6,20.7e6,16.1e6],
                                              [16.1e6,17.8e6,20.5e6,20.5e6]],
                                    [[0.14e6,0.14e6,0.14e6,0.13e6],
                                              [0.14e6,0.14e6,0.14e6,0.13e6]]]), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_Cs = tuned_box_Cs.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 

for nD_config in range(1,5):
    
    for pism_version in ['yes']: #,'no'
        
        for picop_opt in ['no']: #'yes',

            mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt

            for gamma_choice in ['orig','tuned_clara']: 

                if gamma_choice == 'orig':

                    C = C_pico
                    gamma = gT_star_pico
                    
                    if picop_opt in ['2018','2019']:
                        
                        if picop_opt == '2018':
                            gamma_plume = gamma_eff_T_lazero
                            E0 = E0_lazero

                        ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                              C=C, E0=E0, angle_option='appenB',
                                                              box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                              pism_version=pism_version, picop_opt=picop_opt, gamma_plume=gamma_plume)
                    else:
                        ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                      C=C, box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                      pism_version=pism_version, picop_opt=picop_opt)

                    ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                    ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                    
                elif gamma_choice == 'tuned_clara':
                    
                    C = tuned_box_Cs.sel(config=nD_config, pism=pism_version).drop('config').drop('pism')
                    gamma = tuned_box_gammas.sel(config=nD_config, pism=pism_version).drop('config').drop('pism')
                    

                    if picop_opt in ['2018','2019']:
                        
                        if picop_opt == '2018':
                            gamma_plume = gamma_eff_T_lazero
                            E0 = E0_lazero
                        elif picop_opt == '2019':
                            gamma_plume = 0.00019597
                            E0 = 0.0229407
                        
                        ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                              C=C, E0=E0, angle_option='appenB',
                                                              box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                              pism_version=pism_version, picop_opt=picop_opt, gamma_plume=gamma_plume)
                    
                    elif picop_opt == 'no':
                        
                        ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                      C=C, angle_option='appenB', box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                      pism_version=pism_version, picop_opt=picop_opt)
                    
                    else:
                        print('Help! Did not receive PICOP instructuins')
                    
                    ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')
                    ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')

PICOP

In [ ]:
tuned_box_gammas = xr.DataArray(np.array([[[0.39e-5,0.41e-5,0.44e-5,0.39e-5],
                                              [0.39e-5,0.41e-5,0.44e-5,0.39e-5]],
                                         [[0.51e-5,0.73e-5,0.92e-5,0.63e-5],
                                              [0.51e-5,0.73e-5,0.92e-5,0.63e-5]]]
                                        ), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_gammas = tuned_box_gammas.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})


tuned_box_Cs = xr.DataArray(np.array([[[16.5e6,18.0e6,20.7e6,16.1e6],
                                              [16.1e6,17.8e6,20.5e6,20.5e6]],
                                    [[0.14e6,0.14e6,0.14e6,0.13e6],
                                              [0.14e6,0.14e6,0.14e6,0.13e6]]]), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_Cs = tuned_box_Cs.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})

tuned_picop_gammas = xr.DataArray(np.array([[0.94e-4,0.85e-4],
                                          [2.0e-4,1.5e-4]]), dims={'profile_domain': np.array([50,1000]), 'config': range(3,5)})
tuned_picop_gammas = tuned_picop_gammas.assign_coords({'profile_domain': np.array([50,1000]), 'config': range(3,5)})

tuned_picop_E0s = xr.DataArray(np.array([[30.7e-2,34.5e-2],
                                       [95.1e-2,136e-2]]), dims={'profile_domain': np.array([50,1000]), 'config': range(3,5)})
tuned_picop_E0s = tuned_picop_E0s.assign_coords({'profile_domain': np.array([50,1000]), 'config': range(3,5)})

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 

for nD_config in range(3,5):
    
    if nD_config == 4:
        pism_version = 'no'
    elif nD_config == 3:
        pism_version = 'yes'

    mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picopyes'

    for gamma_choice in ['orig','tuned_clara']: 

        if gamma_choice == 'orig':
            picop_opt='2018'
            C = C_pico
            gamma = gT_star_pico

            if picop_opt in ['2018','2019']:

                if picop_opt == '2018':
                    gamma_plume = gamma_eff_T_lazero
                    E0 = E0_lazero

                ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                      C=C, E0=E0, angle_option='appenB',
                                                      box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                      pism_version=pism_version, picop_opt=picop_opt, gamma_plume=gamma_plume)
            else:
                ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                              C=C, box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                              pism_version=pism_version, picop_opt=picop_opt)

            ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
            ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')


        elif gamma_choice == 'tuned_clara':
            picop_opt='2019'
            C = tuned_box_Cs.sel(config=nD_config, pism=pism_version).drop('config').drop('pism')
            gamma = tuned_box_gammas.sel(config=nD_config, pism=pism_version).drop('config').drop('pism')


            if picop_opt in ['2018','2019']:

                if picop_opt == '2018':
                    gamma_plume = gamma_eff_T_lazero
                    E0 = E0_lazero
                elif picop_opt == '2019':
                    gamma_plume = tuned_picop_gammas.sel(config=nD_config).drop('config')
                    E0 = tuned_picop_E0s.sel(config=nD_config).drop('config')

                ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                      C=C, E0=E0, angle_option='appenB',
                                                      box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                      pism_version=pism_version, picop_opt=picop_opt, gamma_plume=gamma_plume)

            elif picop_opt == 'no':

                ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                              C=C, angle_option='appenB', box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                              pism_version=pism_version, picop_opt=picop_opt)

            else:
                print('Help! Did not receive PICOP instructions')

            ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')
            ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS_oneFRIS.nc')